## Trip Advisor Hotel Reviews

### Topic Modeling

#### Important lLibraries

In [1]:
import pandas as pd
import numpy as np
import re
import itertools
import time
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pyLDAvis
import nltk

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
#from sklearn.decomposition import  LatentDirichletAllocation


#from nltk.tokenize import word_tokenize

#import string
#from nltk import PorterStemmer
#from nltk import SnowballStemmer
#from pprint import pprint

import pyLDAvis.gensim_models
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

#### Let's read the data

In [2]:
df1 = pd.read_csv('Trip_df1.csv', index_col=0)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
df1.head()

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Sentiment,Review2
0,Positive,nice hotel expens park good deal hotel anniver...
1,Negative,noth special charg diamond member hilton decid...
3,Positive,uniqu great wonder hotel monaco locat excel sh...
4,Positive,great great seahawk game awesom downfal view b...
5,Positive,love monaco staff husband hotel crazi weekend ...


In [4]:
df1.shape

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(18307, 2)

# (1) Topics Modeling with CountVectorizer

###   CountVectorizer

In [5]:
coun_vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=10000 ,max_df=0.90,min_df = 3)
doc_word2 = coun_vectorizer.fit_transform(df1['Review2'])
pd.DataFrame(doc_word2.toarray(), index=df1.Review2, columns=coun_vectorizer.get_feature_names()).head()

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,abil,abit,abl,abl enjoy,abl leav,abl make,abl room,abl sleep,about,abroad,...,young coupl,young kid,young ladi,young woman,your,yuck,yummi,yunqu,zero,zone
Review2,,,,,,,,,,,,,,,,,,,,,
nice hotel expens park good deal hotel anniversari late even take advic previou review valet park quick easi littl disappoint exist view room room clean nice size comfort wake stiff neck high pillow soundproof like heard music room morn loud bang door open close hear talk hallway mayb noisi neighbor aveda bath product nice goldfish nice touch take advantag longer locat great distanc shop overal nice experi park,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
noth special charg diamond member hilton decid chain shot anniversari seattl start suit pay extra websit descript suit bedroom bathroom standard hotel room take print reserv desk show say like couch desk clerk mix suit descript kimpton websit sorri free breakfast kid embassi suit sit room bathroom bedroom unlik kimpton call suit offer correct fals advertis send kimpton prefer guest websit email failur provid suit advertis websit reserv descript furnish hard copi reserv printout websit desk manag duti repli solut send email trip guest survey follow email mail guess concern guest staff rang indiffer help desk good breakfast spot neighborhood hood hotel best breakfast spot seattl block away conveni hotel exist late insid bellman busi chate cell phone help bag prior email hotel inform anniversari half picki make good nice email say like deliv bottl champagn chocol cover strawberri room celebr foam pillow champagn strawberri foam pillow great room view alley high rise build good good housekeep staff cleaner room properti impress morn shop room short trip bed comfort good heat control inch screen green shine directli eye light sensit tape control thi start hotel clean busi hotel super high rate well chain hotel seattl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
uniqu great wonder hotel monaco locat excel short stroll main downtown shop area friendli room show sign anim hair smell monaco suit sleep area stripe curtain pull close nice touch cosi goldfish name brandi enjoy partak free wine coffe servic lobbi great featur great staff friendli free wireless internet hotel work suit laptop decor love eclect patten color palatt anim print bathrob feel like rock star nice like steril chain hotel hotel person excel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
great great seahawk game awesom downfal view build complain room huge staff help hotel websit seahawk packag charg park voucher taxi problem taxi driver accept voucher bare speak english funni speak arab call start make comment girlfriend cell phone buddi take second realiz say fact speak languag face priceless say larg citi head doorman issu call compani prompli answer apolog offer taxi buck mile stadium game plan taxi return humpin great mind right christma wonder light homeless stow away build entranc leav polic presenc great area stadium activ block pike street waterfront great coffe shop hotel mantain foyer awesom wine tast avail even best take bernard famili safe hotel locat servic desk room bathroom huge jet huge funni hous girlfriend get dress hear knock turn servic scream girlfriend scream floor laugh start talk spanish work place recommend price onlin deal good good besit contain deal voucher travel websit,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
love monaco staff husband hotel crazi weekend attend memori servic best friend husband celebr wed anniversari talk mix emot suit hotel mont carlo loan beauti tan goldfish name joliet weekend visit dog work desk human companion room decor nice couch pillow occitan bath amen welcom sight room quiet peac wireless internet access wonder server morn leav problem print board pas afternoon recept serv oenophil satisfi wine australia scrumptiou cooki restaur close renov final food good drink well word caution restaur large person booth husband tabl smack against stomach coupl inch space mighti uncomfort patro

In [6]:
#By using this function we will be showing all the words that we have in each models.

def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 1- Non-Negative Matrix Factorization (NMF)


In [12]:
# model performing with 2 topic

nmf_model_1 = NMF(2)
doc_topic = nmf_model_1.fit_transform(doc_word2)
display_topics(nmf_model_1, coun_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  0
hotel, room, locat, great, staff, good, breakfast, nice, clean, help, servic, area, small, bathroom, view

Topic  1
beach, room, resort, good, pool, food, great, restaur, like, nice, servic, water, drink, staff, buffet


In [13]:
pyLDAvis.sklearn.prepare(nmf_model_1,doc_word2,coun_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.117118  0.0       1        1  58.590066
1     -0.117118  0.0       2        1  41.409934, topic_info=        Term          Freq         Total Category  logprob  loglift
3948   hotel  50655.000000  50655.000000  Default  30.0000  30.0000
570    beach  10683.000000  10683.000000  Default  29.0000  29.0000
7106  resort  10044.000000  10044.000000  Default  28.0000  28.0000
6478    pool   7689.000000   7689.000000  Default  27.0000  27.0000
4993   locat  10251.000000  10251.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
1393   clean   2764.561169   8361.866338   Topic2  -5.5920  -0.2252
739     best   2354.956500   4573.444588   Topic2  -5.7524   0.2179
9905    work   2242.756337   3897.868861   Topic2  -5.8012   0.3289
5137    love   2291.289338   5821.218479   Topic2  -5.7798  -0.0507
286     area   2144.318341   6623.930936   Topic2  -5.8461  -0.2462

[215 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
59        2  0.999678          activ
79        1  0.985987    addit hotel
135       1  0.990991   airport euro
145       1  0.998178  airport train
150       1  0.994328       alcatraz
...     ...       ...            ...
9740      2  0.731221           week
9838      1  0.948390         window
9838      2  0.051805         window
9905      1  0.424591           work
9905      2  0.575443           work

[231 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [14]:
# model performing with 3 topic

nmf_model_2 = NMF(3)
doc_topic = nmf_model_2.fit_transform(doc_word2)
display_topics(nmf_model_2, coun_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  0
hotel, great, locat, staff, good, breakfast, restaur, help, nice, recommend, clean, citi, area, love, excel

Topic  1
beach, resort, pool, good, food, great, restaur, like, drink, water, nice, servic, buffet, staff, beauti

Topic  2
room, floor, desk, servic, bathroom, nice, small, view, clean, door, good, staff, breakfast, like, shower


In [15]:
pyLDAvis.sklearn.prepare(nmf_model_2,doc_word2,coun_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.142140  0.101528       1        1  37.677275
2      0.029898 -0.157960       2        1  33.460613
1     -0.172038  0.056432       3        1  28.862112, topic_info=        Term          Freq         Total Category  logprob  loglift
7409    room  48310.000000  48310.000000  Default  30.0000  30.0000
3948   hotel  49979.000000  49979.000000  Default  29.0000  29.0000
570    beach   8968.000000   8968.000000  Default  28.0000  28.0000
7106  resort   8346.000000   8346.000000  Default  27.0000  27.0000
3479   great  15844.000000  15844.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
739     best   1954.151124   4285.461710   Topic3  -5.5780   0.4574
6313   place   2427.750596   7312.587803   Topic3  -5.3610   0.1400
8555   staff   2632.536904  12708.977646   Topic3  -5.2800  -0.3317
5137    love   1873.530020   5560.154950   Topic3  -5.6201   0.1548
1393   clean   1734.114923   8470.059548   Topic3  -5.6974  -0.3434

[328 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
26        1  0.994457     accademia
29        2  0.983736   accept room
35        1  1.013311  access hotel
59        3  0.999886         activ
62        3  0.984635    activ like
...     ...       ...           ...
9874      3  0.355287        wonder
9883      1  1.000517  wonder hotel
9905      1  0.044166          work
9905      2  0.607092          work
9905      3  0.348585          work

[441 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

In [16]:
# model performing with 4 topic

nmf_model_3 = NMF(4)
doc_topic = nmf_model_3.fit_transform(doc_word2)
display_topics(nmf_model_3, coun_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  0
hotel, locat, staff, good, breakfast, restaur, area, citi, recommend, help, excel, nice, servic, minut, clean

Topic  1
beach, resort, pool, food, good, restaur, like, water, drink, servic, buffet, nice, beauti, make, take

Topic  2
room, floor, desk, servic, bathroom, nice, view, small, clean, door, good, staff, breakfast, like, shower

Topic  3
great, locat, staff, good, nice, place, clean, love, help, breakfast, friendli, great locat, recommend, excel, littl


In [17]:
pyLDAvis.sklearn.prepare(nmf_model_3,doc_word2,coun_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.091584  0.071215       1        1  29.465789
2     -0.115710  0.137956       2        1  28.488420
1     -0.159088 -0.139537       3        1  22.126497
3      0.183215 -0.069634       4        1  19.919294, topic_info=         Term          Freq         Total Category  logprob  loglift
3948    hotel  46973.000000  46973.000000  Default  30.0000  30.0000
7409     room  44239.000000  44239.000000  Default  29.0000  29.0000
3479    great  29045.000000  29045.000000  Default  28.0000  28.0000
570     beach   7377.000000   7377.000000  Default  27.0000  27.0000
7106   resort   6907.000000   6907.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
1363     citi   1432.171422   4135.920164   Topic4  -5.5179   0.5530
7176  restaur   1697.841549   8552.272586   Topic4  -5.3477  -0.0034
2408    enjoy   1344.530946   3866.694664   Topic4  -5.5810   0.5571
7337    right   1312.040729   4040.943822   Topic4  -5.6055   0.4886
7409     room   1361.236640  44239.959837   Topic4  -5.5687  -1.8678

[442 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
59        3  0.893167        activ
59        4  0.107097        activ
62        3  1.011856   activ like
63        3  0.992731   activ pool
84        1  0.994721  adjac hotel
...     ...       ...          ...
9874      3  0.195802       wonder
9874      4  0.449933       wonder
9905      1  0.072866         work
9905      2  0.600649         work
9905      3  0.326629         work

[675 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4])

In [18]:
# model performing with 5 topic

nmf_model_4 = NMF(5)
doc_topic = nmf_model_4.fit_transform(doc_word2)
display_topics(nmf_model_4, coun_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  0
hotel, locat, staff, breakfast, area, citi, restaur, recommend, help, excel, minut, love, street, servic, station

Topic  1
beach, resort, pool, food, restaur, like, water, drink, servic, buffet, beauti, make, take, vacat, trip

Topic  2
room, floor, desk, servic, view, bathroom, small, door, nice, staff, clean, shower, like, work, give

Topic  3
great, locat, staff, love, place, help, clean, friendli, great locat, nice, recommend, breakfast, wonder, great hotel, hotel great

Topic  4
good, nice, food, breakfast, restaur, clean, place, like, pretti, price, small, drink, excel, servic, buffet


In [19]:
pyLDAvis.sklearn.prepare(nmf_model_4,doc_word2,coun_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.094679 -0.092485       1        1  25.998734
2     -0.123140 -0.150996       2        1  25.621783
1     -0.167942  0.050951       3        1  18.243431
3      0.208459  0.004718       4        1  16.741453
4     -0.012055  0.187813       5        1  13.394598, topic_info=          Term          Freq         Total Category  logprob  loglift
3948     hotel  44603.000000  44603.000000  Default  30.0000  30.0000
3479     great  29153.000000  29153.000000  Default  29.0000  29.0000
3324      good  22147.000000  22147.000000  Default  28.0000  28.0000
7409      room  42392.000000  42392.000000  Default  27.0000  27.0000
570      beach   6620.000000   6620.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
6478      pool    972.966014   6159.735460   Topic5  -5.5076   0.1649
286       area    969.347000   6451.384305   Topic5  -5.5114   0.1149
3137  friendli    923.540080   6226.481196   Topic5  -5.5598   0.1020
528   bathroom    899.872875   5416.542218   Topic5  -5.5857   0.2154
8133      shop    749.587947   3985.901928   Topic5  -5.7685   0.3393

[505 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
59        3  0.848119        activ
59        4  0.152084        activ
60        3  1.002002  activ beach
153       1  0.995678   alessandra
176       1  0.196282         amaz
...     ...       ...          ...
9874      4  0.475838       wonder
9905      1  0.044713         work
9905      2  0.552300         work
9905      3  0.272213         work
9905      5  0.130482         work

[890 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4, 5])

In [20]:
# model performing with 6 topic

nmf_model_5 = NMF(6)
doc_topic = nmf_model_5.fit_transform(doc_word2)
display_topics(nmf_model_5, coun_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  0
hotel, area, citi, servic, locat, restaur, star, like, price, minut, travel, recommend, breakfast, park, street

Topic  1
beach, resort, pool, food, restaur, water, like, drink, servic, buffet, beauti, make, take, vacat, staff

Topic  2
room, floor, desk, servic, view, bathroom, small, nice, door, like, work, shower, say, give, problem

Topic  3
great, nice, place, room, locat, love, great locat, great hotel, hotel great, view, food, great place, littl, best, servic

Topic  4
good, nice, food, breakfast, restaur, clean, like, place, pretti, servic, drink, price, room, small, buffet

Topic  5
staff, locat, help, friendli, breakfast, clean, excel, love, recommend, place, comfort, wonder, restaur, street, citi


In [21]:
pyLDAvis.sklearn.prepare(nmf_model_5,doc_word2,coun_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.157445  0.050554       1        1  22.358450
0     -0.112631 -0.165000       2        1  21.643739
5      0.014699 -0.181083       3        1  16.255871
1     -0.035261  0.147176       4        1  16.142115
3      0.292806 -0.008312       5        1  12.234210
4     -0.002168  0.156665       6        1  11.365614, topic_info=          Term          Freq         Total Category  logprob  loglift
3948     hotel  52038.000000  52038.000000  Default  30.0000  30.0000
3479     great  36595.000000  36595.000000  Default  29.0000  29.0000
3324      good  21759.000000  21759.000000  Default  28.0000  28.0000
7409      room  42564.000000  42564.000000  Default  27.0000  27.0000
8555     staff  11635.000000  11635.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
8264      size    719.427237   2443.939388   Topic6  -5.6453   0.9517
2617    expens    663.134415   2016.365180   Topic6  -5.7268   1.0625
7409      room   1151.694207  42564.073176   Topic6  -5.1747  -1.4352
286       area    727.543074   6442.463839   Topic6  -5.6341  -0.0064
528   bathroom    685.042983   5134.051263   Topic6  -5.6943   0.1604

[577 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
38        3  1.019695  access metro
59        4  0.796505         activ
59        5  0.200576         activ
59        6  0.003478         activ
60        4  1.013641   activ beach
...     ...       ...           ...
9905      1  0.548566          work
9905      2  0.034705          work
9905      3  0.035580          work
9905      4  0.244974          work
9905      6  0.136194          work

[1051 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 6, 2, 4, 5])

### 2- LDA (Latent Dirichlet Allocation)

In [22]:
# model performing with 2 topic
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

lda1 = LatentDirichletAllocation(2)
doc_topic_ld = lda1.fit_transform(doc_word2)
display_topics(lda1, coun_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic  0
hotel, room, great, locat, staff, good, breakfast, nice, clean, help, place, servic, friendli, small, recommend

Topic  1
room, beach, resort, food, pool, good, great, restaur, servic, like, staff, hotel, nice, place, water


In [23]:
pyLDAvis.sklearn.prepare(lda1,doc_word2,coun_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.101696  0.0       1        1  59.551659
1     -0.101696  0.0       2        1  40.448341, topic_info=        Term          Freq         Total Category  logprob  loglift
3948   hotel  48031.000000  48031.000000  Default  30.0000  30.0000
570    beach   9132.000000   9132.000000  Default  29.0000  29.0000
7106  resort   7928.000000   7928.000000  Default  28.0000  28.0000
6478    pool   7521.000000   7521.000000  Default  27.0000  27.0000
4993   locat  12150.000000  12150.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
2408   enjoy   2390.390923   4338.205765   Topic2  -5.7140   0.3091
4924   littl   2534.261028   5435.517899   Topic2  -5.6555   0.1421
1393   clean   2851.180283   9684.823450   Topic2  -5.5377  -0.3177
3948   hotel   4414.279438  48031.634904   Topic2  -5.1006  -1.4819
5137    love   2511.082152   7056.669882   Topic2  -5.6647  -0.1281

[191 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
59        1  0.027993      activ
59        2  0.971873      activ
210       1  0.999629  amsterdam
210       2  0.000940  amsterdam
286       1  0.694236       area
...     ...       ...        ...
9740      2  0.715109       week
9838      1  0.967120     window
9838      2  0.032713     window
9905      1  0.385676       work
9905      2  0.614305       work

[290 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [24]:
# model performing with 3 topic

lda2 = LatentDirichletAllocation(3)
doc_topic_ld = lda2.fit_transform(doc_word2)
display_topics(lda2, coun_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic  0
room, resort, beach, good, food, pool, like, servic, hotel, nice, water, restaur, great, drink, staff

Topic  1
hotel, great, staff, love, servic, room, pool, wonder, food, beauti, restaur, place, excel, best, friendli

Topic  2
hotel, room, locat, great, good, staff, breakfast, nice, clean, help, small, bathroom, area, comfort, friendli


In [25]:
pyLDAvis.sklearn.prepare(lda2,doc_word2,coun_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.125895 -0.042912       1        1  49.687129
0     -0.107898 -0.068682       2        1  33.579757
1     -0.017997  0.111594       3        1  16.733114, topic_info=        Term          Freq         Total Category  logprob  loglift
3948   hotel  47991.000000  47991.000000  Default  30.0000  30.0000
3479   great  19805.000000  19805.000000  Default  29.0000  29.0000
7106  resort   7924.000000   7924.000000  Default  28.0000  28.0000
570    beach   9137.000000   9137.000000  Default  27.0000  27.0000
5137    love   7091.000000   7091.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
3781    help   1416.426024   7548.515373   Topic3  -5.3546   0.1146
5265    make   1182.893325   5098.894471   Topic3  -5.5348   0.3267
3324    good   1498.825389  15928.376325   Topic3  -5.2981  -0.5756
5734    nice   1218.740241  11735.669217   Topic3  -5.5050  -0.4770
1393   clean   1198.205429   9675.591748   Topic3  -5.5220  -0.3010

[299 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
10        1  0.206921      absolut
10        2  0.263470      absolut
10        3  0.529511      absolut
176       1  0.127222         amaz
176       2  0.195554         amaz
...     ...       ...          ...
9939      2  0.016415  worth penni
9939      3  0.984871  worth penni
9965      1  0.258772         year
9965      2  0.341047         year
9965      3  0.400107         year

[522 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

In [26]:
# model performing with 4 topic

lda3 = LatentDirichletAllocation(4)
doc_topic_ld = lda3.fit_transform(doc_word2)
display_topics(lda3, coun_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic  0
resort, beach, room, food, good, great, pool, restaur, like, water, staff, drink, nice, servic, buffet

Topic  1
room, hotel, servic, desk, like, staff, floor, good, nice, place, bathroom, small, door, park, work

Topic  2
hotel, room, great, locat, staff, good, breakfast, clean, nice, help, friendli, excel, recommend, love, comfort

Topic  3
great, pool, hotel, beach, restaur, room, food, servic, love, nice, staff, place, good, beauti, view


In [27]:
pyLDAvis.sklearn.prepare(lda3,doc_word2,coun_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.116265  0.084560       1        1  38.669102
1      0.090983 -0.111636       2        1  24.201418
0     -0.125998 -0.056438       3        1  23.647902
3     -0.081249  0.083514       4        1  13.481579, topic_info=           Term          Freq         Total Category  logprob  loglift
570       beach   9173.000000   9173.000000  Default  30.0000  30.0000
3948      hotel  47896.000000  47896.000000  Default  29.0000  29.0000
7106     resort   7966.000000   7966.000000  Default  28.0000  28.0000
6478       pool   7556.000000   7556.000000  Default  27.0000  27.0000
3479      great  19692.000000  19692.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
899   breakfast   1049.465100   8939.316908   Topic4  -5.4384  -0.1383
7106     resort    987.720725   7966.991610   Topic4  -5.4991  -0.0838
2222      drink    872.768627   4199.675619   Topic4  -5.6228   0.4328
6957  recommend    900.466409   6033.932925   Topic4  -5.5916   0.1016
1393      clean    913.328225   9652.857802   Topic4  -5.5774  -0.3541

[386 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
59        1  0.035718      activ
59        2  0.042687      activ
59        3  0.766620      activ
59        4  0.155066      activ
176       1  0.268158       amaz
...     ...       ...        ...
9905      3  0.359655       work
9905      4  0.062290       work
9909      3  0.987599  work hard
9909      4  0.011711  work hard
9997      4  0.979937      yunqu

[757 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4])

In [28]:
# model performing with 5 topic

lda4 = LatentDirichletAllocation(5)
doc_topic_ld = lda4.fit_transform(doc_word2)
display_topics(lda4, coun_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic  0
hotel, room, locat, good, staff, great, breakfast, excel, clean, help, recommend, citi, station, friendli, restaur

Topic  1
resort, beach, food, room, great, restaur, good, pool, vacat, servic, like, staff, beauti, drink, trip

Topic  2
hotel, room, great, locat, nice, staff, breakfast, clean, place, help, good, comfort, love, friendli, small

Topic  3
pool, great, beach, good, food, room, hotel, nice, restaur, love, staff, like, clean, drink, place

Topic  4
room, hotel, servic, desk, like, staff, good, door, place, work, manag, say, floor, call, bathroom


In [29]:
pyLDAvis.sklearn.prepare(lda4,doc_word2,coun_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.116082 -0.018837       1        1  23.563720
0     -0.131000 -0.058861       2        1  23.479824
4     -0.014061  0.151329       3        1  20.779694
1      0.160526 -0.006027       4        1  19.843210
3      0.100618 -0.067603       5        1  12.333552, topic_info=          Term          Freq         Total Category  logprob  loglift
7106    resort   7991.000000   7991.000000  Default  30.0000  30.0000
3948     hotel  48005.000000  48005.000000  Default  29.0000  29.0000
570      beach   9132.000000   9132.000000  Default  28.0000  28.0000
6478      pool   7488.000000   7488.000000  Default  27.0000  27.0000
3479     great  19666.000000  19666.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
9548      view    840.197970   5395.125896   Topic5  -5.5718   0.2332
286       area    881.220947   6246.286344   Topic5  -5.5242   0.1344
3137  friendli    853.420132   6306.522520   Topic5  -5.5562   0.0928
9740      week    673.894410   3149.603332   Topic5  -5.7924   0.5509
5265      make    706.256180   5099.923002   Topic5  -5.7455   0.1158

[481 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
59        1  0.054896  activ
59        2  0.003485  activ
59        3  0.036598  activ
59        4  0.731951  activ
59        5  0.173403  activ
...     ...       ...    ...
9984      1  0.710830   york
9984      2  0.170886   york
9984      3  0.047389   york
9984      4  0.067493   york
9984      5  0.002872   york

[1065 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 5, 2, 4])

In [30]:
# model performing with 6 topic

lda5 = LatentDirichletAllocation(6)
doc_topic_ld = lda5.fit_transform(doc_word2)
display_topics(lda5, coun_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic  0
hotel, room, servic, staff, great, locat, best, excel, recommend, citi, help, trip, restaur, travel, view

Topic  1
room, hotel, desk, like, servic, floor, bathroom, door, staff, good, work, nice, small, say, shower

Topic  2
resort, beach, food, great, room, good, pool, restaur, like, staff, nice, drink, water, servic, beauti

Topic  3
hotel, room, locat, great, good, staff, breakfast, clean, nice, help, friendli, small, love, excel, recommend

Topic  4
hotel, pool, good, restaur, food, room, breakfast, servic, area, club, place, nice, taxi, bali, drink

Topic  5
great, room, view, hotel, nice, place, love, locat, park, staff, suit, area, restaur, friendli, wonder


In [31]:
pyLDAvis.sklearn.prepare(lda5,doc_word2,coun_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.156857  0.040327       1        1  24.954305
3      0.122065 -0.015447       2        1  24.688243
1     -0.056542 -0.164282       3        1  19.284426
0      0.086250 -0.017142       4        1  13.338775
5      0.075070  0.078292       5        1  10.233982
4     -0.069987  0.078252       6        1   7.500269, topic_info=        Term          Freq         Total Category  logprob  loglift
3948   hotel  48032.000000  48032.000000  Default  30.0000  30.0000
7106  resort   7867.000000   7867.000000  Default  29.0000  29.0000
6478    pool   7572.000000   7572.000000  Default  28.0000  28.0000
3479   great  19717.000000  19717.000000  Default  27.0000  27.0000
570    beach   9090.000000   9090.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
6614   price    508.454755   4853.654168   Topic6  -5.5767   0.3341
5137    love    544.769166   7047.460055   Topic6  -5.5077   0.0302
4865    like    534.540518   8302.181820   Topic6  -5.5267  -0.1526
8555   staff    574.791282  15050.140134   Topic6  -5.4541  -0.6749
3479   great    506.305306  19717.155710   Topic6  -5.5809  -1.0719

[546 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
59        1  0.832865      activ
59        2  0.020228      activ
59        3  0.034300      activ
59        4  0.016710      activ
59        5  0.044853      activ
...     ...       ...        ...
9984      5  0.098522       york
9984      6  0.001428       york
9985      2  0.027590  york citi
9985      4  0.924262  york citi
9985      5  0.041385  york citi

[1375 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1, 6, 5])

## 3- CorEx model 

CorEx looks for a group of words that are both as broad (in many documents) and overlapping as possible (both together will lead to high TC)

Then, you look for another set of words (not overlapping with your previous group) with as high a TC as possible



In [32]:
# Install CorEx 
#!pip install corextopic
#!pip install networkx

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
words = list(np.asarray(coun_vectorizer.get_feature_names()))

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
from corextopic import corextopic as ct
topic_model = ct.Corex(n_hidden=6, words=words, seed=1)
topic_model.fit(doc_word2, words=words, docs=df1['Review2'])

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
# Print all topics from the CorEx topic model
topics = topic_model.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: beach,resort,food,pool,cana,punta,punta cana,vacat,drink,beauti
1: water,like,take,work,make,say,problem,tri,wait,towel
2: desk,call,servic,reserv,morn,manag,guest,person,give,charg
3: fruit,shower,coffe,open,fresh,good,juic,small,cereal,door
4: locat,station,hotel,street,metro,citi,help,central,great locat,hotel locat
5: floor,bathroom,view,internet,free,breakfast,room,area,larg,access


/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 4- Latent Semantic Analysis (LSA) 


 Note that: Latent Semantic Analysis (LSA) is just another name for Signular Value Decomposition (SVD) 
applied to Natural Language Processing (NLP)

In [10]:
# model performing with 5 topic
lsa4 = TruncatedSVD(5)
doc_topic = lsa4.fit_transform(doc_word2)
display_topics(lsa4, coun_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic  0
hotel, room, great, good, staff, beach, nice, restaur, servic, resort, pool, food, like, locat, clean

Topic  1
resort, beach, pool, food, good, drink, restaur, water, like, buffet, vacat, servic, beauti, great, make

Topic  2
room, desk, floor, bathroom, door, view, shower, small, window, room room, smoke, say, room servic, request, give

Topic  3
hotel, resort, beach, pool, guest, water, say, servic, like, manag, star, reserv, call, take, punta

Topic  4
great, resort, staff, hotel, beach, room, love, wonder, beauti, servic, guest, return, vacat, desk, manag


# (2) Topics Modeling with Tfidf Vectorizer

### Tfidf Vectorizer

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

Tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=10000, max_df=0.90,min_df = 3)
doc_word2 = Tfidf_vectorizer.fit_transform(df1['Review2'])
pd.DataFrame(doc_word2.toarray(), index=df1.Review2, columns=Tfidf_vectorizer.get_feature_names()).head()

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,abil,abit,abl,abl enjoy,abl leav,abl make,abl room,abl sleep,about,abroad,...,young coupl,young kid,young ladi,young woman,your,yuck,yummi,yunqu,zero,zone
Review2,,,,,,,,,,,,,,,,,,,,,
nice hotel expens park good deal hotel anniversari late even take advic previou review valet park quick easi littl disappoint exist view room room clean nice size comfort wake stiff neck high pillow soundproof like heard music room morn loud bang door open close hear talk hallway mayb noisi neighbor aveda bath product nice goldfish nice touch take advantag longer locat great distanc shop overal nice experi park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
noth special charg diamond member hilton decid chain shot anniversari seattl start suit pay extra websit descript suit bedroom bathroom standard hotel room take print reserv desk show say like couch desk clerk mix suit descript kimpton websit sorri free breakfast kid embassi suit sit room bathroom bedroom unlik kimpton call suit offer correct fals advertis send kimpton prefer guest websit email failur provid suit advertis websit reserv descript furnish hard copi reserv printout websit desk manag duti repli solut send email trip guest survey follow email mail guess concern guest staff rang indiffer help desk good breakfast spot neighborhood hood hotel best breakfast spot seattl block away conveni hotel exist late insid bellman busi chate cell phone help bag prior email hotel inform anniversari half picki make good nice email say like deliv bottl champagn chocol cover strawberri room celebr foam pillow champagn strawberri foam pillow great room view alley high rise build good good housekeep staff cleaner room properti impress morn shop room short trip bed comfort good heat control inch screen green shine directli eye light sensit tape control thi start hotel clean busi hotel super high rate well chain hotel seattl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
uniqu great wonder hotel monaco locat excel short stroll main downtown shop area friendli room show sign anim hair smell monaco suit sleep area stripe curtain pull close nice touch cosi goldfish name brandi enjoy partak free wine coffe servic lobbi great featur great staff friendli free wireless internet hotel work suit laptop decor love eclect patten color palatt anim print bathrob feel like rock star nice like steril chain hotel hotel person excel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
great great seahawk game awesom downfal view build complain room huge staff help hotel websit seahawk packag charg park voucher taxi problem taxi driver accept voucher bare speak english funni speak arab call start make comment girlfriend cell phone buddi take second realiz say fact speak languag face priceless say larg citi head doorman issu call compani prompli answer apolog offer taxi buck mile stadium game plan taxi return humpin great mind right christma wonder light homeless stow away build entranc leav polic presenc great area stadium activ block pike street waterfront great coffe shop hotel mantain foyer awesom wine tast avail even best take bernard famili safe hotel locat servic desk room bathroom huge jet huge funni hous girlfriend get dress hear knock turn servic scream girlfriend scream floor laugh start talk spanish work place recommend price onlin deal good good besit contain deal voucher travel websit,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
love monaco staff husband hotel crazi weekend attend memori servic best friend husband celebr wed anniversari talk mix emot suit hotel mont carlo loan beauti tan goldfish name joliet weekend visit dog work desk human companion room decor nice couch pillow occitan bath amen welcom sight room quiet peac wireless internet access wonder server morn leav problem print board pas afternoon recept serv oenophil satisfi wine australia scrumptiou cooki r

### 1- Non-Negative Matrix Factorization (NMF)


In [41]:
# model performing with 2 topic

nmf_model1 = NMF(2)
doc_topic = nmf_model1.fit_transform(doc_word2)
display_topics(nmf_model1, Tfidf_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  0
hotel, room, great, locat, staff, good, breakfast, nice, clean, help, excel, friendli, small, recommend, love

Topic  1
resort, beach, pool, food, good, water, drink, like, restaur, room, great, vacat, beauti, servic, buffet


In [42]:
pyLDAvis.sklearn.prepare(nmf_model1,doc_word2,Tfidf_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.132471  0.0       1        1  66.198987
1     -0.132471  0.0       2        1  33.801013, topic_info=         Term         Freq        Total Category  logprob  loglift
7106   resort   490.000000   490.000000  Default  30.0000  30.0000
570     beach   440.000000   440.000000  Default  29.0000  29.0000
3948    hotel  1158.000000  1158.000000  Default  28.0000  28.0000
6478     pool   325.000000   325.000000  Default  27.0000  27.0000
2955     food   338.000000   338.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
739      best   111.964471   249.802874   Topic2  -6.0589   0.2822
7409     room   163.841245   915.075294   Topic2  -5.6782  -0.6354
5734     nice   132.647981   454.942445   Topic2  -5.8894  -0.1478
6681  problem   102.424460   197.571766   Topic2  -6.1479   0.4277
4924    littl   104.298210   255.708148   Topic2  -6.1298   0.1879

[204 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
31        1  1.007379  access avail
59        2  0.998813         activ
65        2  1.043240       activit
73        1  1.055984       ad bonu
79        1  1.027883   addit hotel
...     ...       ...           ...
9681      1  0.194983         water
9681      2  0.803145         water
9740      1  0.330101          week
9740      2  0.672206          week
9838      1  0.996868        window

[217 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [43]:
# model performing with 3 topic

nmf_model2 = NMF(3)
doc_topic = nmf_model2.fit_transform(doc_word2)
display_topics(nmf_model2, Tfidf_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  0
room, hotel, floor, nice, servic, small, desk, bathroom, good, like, view, park, door, shower, area

Topic  1
resort, beach, food, pool, great, drink, beauti, vacat, good, restaur, water, like, buffet, cana, punta

Topic  2
hotel, great, locat, staff, help, excel, friendli, recommend, love, clean, breakfast, good, great locat, citi, room


In [44]:
pyLDAvis.sklearn.prepare(nmf_model2,doc_word2,Tfidf_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.196949  0.072442       1        1  39.150552
0     -0.002356 -0.142345       2        1  35.801573
1      0.199305  0.069903       3        1  25.047875, topic_info=        Term         Freq        Total Category  logprob  loglift
7106  resort   441.000000   441.000000  Default  30.0000  30.0000
570    beach   393.000000   393.000000  Default  29.0000  29.0000
3948   hotel  1245.000000  1245.000000  Default  28.0000  28.0000
4993   locat   539.000000   539.000000  Default  27.0000  27.0000
3479   great   749.000000   749.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
8973    take    82.376327   188.105318   Topic3  -6.0661   0.5587
5734    nice    93.505448   445.318397   Topic3  -5.9393  -0.1764
5137    love    89.036670   357.831756   Topic3  -5.9883  -0.0066
4924   littl    82.493717   251.609592   Topic3  -6.0646   0.2692
8555   staff    83.442508   500.843993   Topic3  -6.0532  -0.4078

[322 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
26        1  1.033242     accademia
38        1  0.918269  access metro
59        1  0.013805         activ
59        3  0.980171         activ
60        3  0.930782   activ beach
...     ...       ...           ...
9874      2  0.008139        wonder
9874      3  0.305201        wonder
9905      2  0.627525          work
9905      3  0.370538          work
9926      2  0.986912          worn

[425 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

In [45]:
# model performing with 4 topic

nmf_model3 = NMF(4)
doc_topic = nmf_model3.fit_transform(doc_word2)
display_topics(nmf_model3, Tfidf_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  0
room, hotel, floor, nice, desk, small, servic, bathroom, view, like, good, shower, door, park, area

Topic  1
resort, beach, food, pool, drink, beauti, vacat, restaur, good, water, like, buffet, great, cana, punta

Topic  2
great, locat, great locat, hotel great, great hotel, nice, room, place, staff, locat great, clean, hotel great locat, friendli, great place, valu

Topic  3
hotel, excel, staff, locat, recommend, help, love, breakfast, citi, good, friendli, station, clean, wonder, perfect


In [46]:
pyLDAvis.sklearn.prepare(nmf_model3,doc_word2,Tfidf_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.079419 -0.174498       1        1  32.007587
0      0.073691  0.025759       2        1  28.595594
1      0.234595  0.039453       3        1  22.579791
2     -0.228868  0.109286       4        1  16.817028, topic_info=             Term         Freq        Total Category  logprob  loglift
3479        great  1046.000000  1046.000000  Default  30.0000  30.0000
3549  great locat   354.000000   354.000000  Default  29.0000  29.0000
4993        locat   649.000000   649.000000  Default  28.0000  28.0000
7106       resort   405.000000   405.000000  Default  27.0000  27.0000
3948        hotel  1211.000000  1211.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
6614        price    87.885748   258.918682   Topic4  -5.6029   0.7023
1584      comfort    84.438884   268.075785   Topic4  -5.6429   0.6275
3324         good   101.924012   539.670436   Topic4  -5.4547   0.1160
7337        right    76.190864   209.637543   Topic4  -5.7457   0.7706
899     breakfast    80.633135   376.264047   Topic4  -5.6890   0.2424

[399 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
26        1  0.958070     accademia
29        2  0.880700   accept room
59        3  0.961876         activ
59        4  0.029596         activ
60        3  1.013074   activ beach
...     ...       ...           ...
9874      4  0.139281        wonder
9883      1  0.989023  wonder hotel
9905      2  0.624101          work
9905      3  0.371805          work
9926      2  0.991054          worn

[583 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3])

In [47]:
# model performing with 5 topic

nmf_model4 = NMF(5)
doc_topic = nmf_model4.fit_transform(doc_word2)
display_topics(nmf_model4, Tfidf_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "



Topic  0
room, hotel, desk, floor, servic, view, like, small, bathroom, door, park, nice, work, shower, say

Topic  1
resort, beach, food, pool, good, drink, vacat, restaur, beauti, water, buffet, like, great, cana, punta

Topic  2
great, great locat, locat, hotel great, great hotel, locat great, room, hotel great locat, place, staff, great place, nice, clean, friendli, great hotel great

Topic  3
hotel, love, staff, recommend, wonder, excel, help, perfect, friendli, best, recommend hotel, fantast, return, florenc, highli

Topic  4
good, hotel, breakfast, locat, station, nice, clean, valu, minut, room, euro, citi, street, metro, excel


In [48]:
pyLDAvis.sklearn.prepare(nmf_model4,doc_word2,Tfidf_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.016037  0.208140       1        1  23.417991
0      0.112853 -0.074759       2        1  22.223393
1      0.238528 -0.056111       3        1  20.241534
4     -0.094183  0.044388       4        1  20.232390
2     -0.241161 -0.121658       5        1  13.884692, topic_info=             Term         Freq        Total Category  logprob  loglift
3479        great  1059.000000  1059.000000  Default  30.0000  30.0000
3549  great locat   360.000000   360.000000  Default  29.0000  29.0000
3948        hotel  1279.000000  1279.000000  Default  28.0000  28.0000
4993        locat   644.000000   644.000000  Default  27.0000  27.0000
7106       resort   375.000000   375.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
810         block    65.266740   149.343325   Topic5  -5.7089   1.1466
7337        right    70.347605   205.255348   Topic5  -5.6339   0.9036
1955      definit    70.229391   210.226173   Topic5  -5.6356   0.8780
1584      comfort    67.403879   270.146029   Topic5  -5.6766   0.5861
6614        price    66.671095   249.961754   Topic5  -5.6876   0.6528

[475 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
18        1  0.968222  absolut love hotel
59        1  0.031193               activ
59        3  0.935797               activ
59        5  0.031193               activ
61        3  1.001298           activ kid
...     ...       ...                 ...
9905      4  0.028528                work
9909      3  1.023326           work hard
9965      1  0.538547                year
9965      2  0.158396                year
9965      3  0.310456                year

[769 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 5, 3])

### 2- LDA (Latent Dirichlet Allocation)

In [49]:
# model performing with 2 topic

lda_1 = LatentDirichletAllocation(2)
doc_topic_ld = lda_1.fit_transform(doc_word2)
display_topics(lda_1, Tfidf_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic  0
beach, resort, pool, food, room, servic, like, good, great, water, place, restaur, beauti, drink, nice

Topic  1
hotel, room, great, locat, staff, good, breakfast, nice, clean, help, excel, friendli, comfort, love, recommend


In [50]:
pyLDAvis.sklearn.prepare(lda_1,doc_word2,Tfidf_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=             x    y  topics  cluster       Freq
topic                                          
1      0.10249  0.0       1        1  59.425217
0     -0.10249  0.0       2        1  40.574783, topic_info=        Term        Freq        Total Category  logprob  loglift
570    beach  334.000000   334.000000  Default  30.0000  30.0000
7106  resort  318.000000   318.000000  Default  29.0000  29.0000
6478    pool  284.000000   284.000000  Default  28.0000  28.0000
4993   locat  455.000000   455.000000  Default  27.0000  27.0000
2955    food  305.000000   305.000000  Default  26.0000  26.0000
...      ...         ...          ...      ...      ...      ...
5734    nice  137.452347   429.039172   Topic2  -6.0364  -0.2362
8555   staff  127.986642   464.017682   Topic2  -6.1078  -0.3860
3948   hotel  137.310335  1124.955418   Topic2  -6.0375  -1.2012
2408   enjoy  103.837717   206.824725   Topic2  -6.3169   0.2130
7287  review  100.744930   213.525942   Topic2  -6.3471   0.1509

[173 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
59        1  0.014909      activ
59        2  0.983961      activ
210       1  0.993884  amsterdam
210       2  0.010353  amsterdam
286       1  0.711247       area
...     ...       ...        ...
9681      2  0.852835      water
9740      1  0.283523       week
9740      2  0.711957       week
9905      1  0.308561       work
9905      2  0.694263       work

[253 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [51]:
# model performing with 3 topic

lda_2 = LatentDirichletAllocation(3)
doc_topic_ld = lda_2.fit_transform(doc_word2)
display_topics(lda_2, Tfidf_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic  0
dirti, smoke, smell, carpet, stain, bad, room, wall, rude, smoke room, terribl, dump, horribl, sheet, star

Topic  1
beach, resort, pool, food, room, great, good, servic, like, restaur, beauti, water, vacat, drink, place

Topic  2
hotel, room, great, locat, staff, good, nice, breakfast, clean, help, excel, love, friendli, place, servic


In [52]:
pyLDAvis.sklearn.prepare(lda_2,doc_word2,Tfidf_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.047343  0.104891       1        1  64.126810
1     -0.084385 -0.086912       2        1  30.507700
0      0.131728 -0.017978       3        1   5.365489, topic_info=            Term        Freq        Total Category  logprob  loglift
570        beach  332.000000   332.000000  Default  30.0000  30.0000
7106      resort  316.000000   316.000000  Default  29.0000  29.0000
8359       smoke   84.000000    84.000000  Default  28.0000  28.0000
6478        pool  280.000000   280.000000  Default  27.0000  27.0000
2090       dirti   76.000000    76.000000  Default  26.0000  26.0000
...          ...         ...          ...      ...      ...      ...
8093      shabbi   11.941465    19.251558   Topic3  -6.4565   2.4476
7409        room   30.437318   876.445610   Topic3  -5.5209  -0.4350
3693     hallway   13.640651    33.923969   Topic3  -6.3235   2.0141
3948       hotel   17.979635  1112.730216   Topic3  -6.0473  -1.2001
8695  star hotel   12.789962    72.379358   Topic3  -6.3879   1.1919

[227 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
59        1  0.105152           activ
59        2  0.886284           activ
110       3  0.913088  affinia dumont
210       1  0.985635       amsterdam
210       3  0.010485       amsterdam
...     ...       ...             ...
9838      1  0.884545          window
9838      2  0.018052          window
9838      3  0.099286          window
9874      1  0.690596          wonder
9874      2  0.307427          wonder

[358 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [53]:
# model performing with 4 topic

lda_3 = LatentDirichletAllocation(4)
doc_topic_ld = lda_3.fit_transform(doc_word2)
display_topics(lda_3, Tfidf_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic  0
hotel, great, locat, london, staff, excel, friendli, clean, help, great hotel, great locat, hotel great, valu, staff friendli, friendli help

Topic  1
hotel, wonder, staff, servic, love, perfect, great, beauti, room, best, recommend, experi, help, locat, fantast

Topic  2
hotel, room, great, locat, good, nice, breakfast, staff, clean, small, help, bathroom, place, view, comfort

Topic  3
beach, resort, pool, food, room, great, good, servic, like, restaur, beauti, vacat, water, drink, place


In [54]:
pyLDAvis.sklearn.prepare(lda_3,doc_word2,Tfidf_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.009830  0.024360       1        1  59.084744
3      0.146449  0.025942       2        1  28.936741
1     -0.022624 -0.101714       3        1   7.371813
0     -0.113996  0.051413       4        1   4.606703, topic_info=            Term         Freq        Total Category  logprob  loglift
3948       hotel  1137.000000  1137.000000  Default  30.0000  30.0000
4993       locat   464.000000   464.000000  Default  29.0000  29.0000
3479       great   632.000000   632.000000  Default  28.0000  28.0000
8555       staff   469.000000   469.000000  Default  27.0000  27.0000
570        beach   325.000000   325.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
3324        good    19.149213   502.095174   Topic4  -5.8318  -0.1889
5137        love    17.403747   326.568681   Topic4  -5.9274   0.1457
8606  staff help    15.014995   100.871384   Topic4  -6.0750   1.1729
7409        room    17.337502   881.190943   Topic4  -5.9312  -0.8507
5734        nice    15.312325   428.818860   Topic4  -6.0554  -0.2547

[329 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
10        1  0.410952        absolut
10        2  0.352245        absolut
10        3  0.225045        absolut
10        4  0.009785        absolut
30        1  0.888996         access
...     ...       ...            ...
9874      2  0.288075         wonder
9874      3  0.257522         wonder
9874      4  0.026189         wonder
9897      2  0.166147  wonder wonder
9897      3  0.830736  wonder wonder

[627 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])

In [55]:
# model performing with 5 topic

lda_4 = LatentDirichletAllocation(5)
doc_topic_ld = lda_4.fit_transform(doc_word2)
display_topics(lda_4, Tfidf_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic  0
hotel, great, room, locat, servic, love, staff, pari, place, nice, wonder, best, view, comfort, perfect

Topic  1
hotel, great, locat, room, staff, help, excel, clean, good, friendli, breakfast, recommend, love, nice, great locat

Topic  2
hotel, room, good, nice, servic, breakfast, great, floor, staff, locat, small, place, like, bathroom, area

Topic  3
bali, hong, kong, hong kong, waikiki, villa, ubud, beach, view, kuta, kowloon, pool, ocean, harbour, nusa

Topic  4
resort, beach, food, pool, great, good, vacat, beauti, drink, restaur, room, like, water, servic, nice


In [56]:
pyLDAvis.sklearn.prepare(lda_4,doc_word2,Tfidf_vectorizer)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.033485 -0.039993       1        1  42.806343
1     -0.134691  0.028608       2        1  22.882238
4      0.096893 -0.120315       3        1  22.779149
0     -0.061019  0.016825       4        1   8.598271
3      0.132303  0.114876       5        1   2.933999, topic_info=           Term         Freq        Total Category  logprob  loglift
570       beach   335.000000   335.000000  Default  30.0000  30.0000
7106     resort   315.000000   315.000000  Default  29.0000  29.0000
3948      hotel  1124.000000  1124.000000  Default  28.0000  28.0000
6478       pool   283.000000   283.000000  Default  27.0000  27.0000
4993      locat   455.000000   455.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
570       beach    19.658527   335.045194   Topic5  -5.3544   0.6930
9548       view    17.272202   256.717007   Topic5  -5.4838   0.8299
6478       pool    15.196130   283.210530   Topic5  -5.6119   0.6037
3919  honeymoon    11.529456    73.161201   Topic5  -5.8880   1.6810
9575  view room     8.765885    54.610489   Topic5  -6.1621   1.6994

[399 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
26        2  0.943443  accademia
59        1  0.120583      activ
59        2  0.060292      activ
59        3  0.798863      activ
59        4  0.015073      activ
...     ...       ...        ...
9874      5  0.013274     wonder
9905      1  0.576888       work
9905      2  0.048074       work
9905      3  0.330509       work
9905      4  0.042065       work

[822 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 5, 1, 4])

## 3- CorEx model 

In [57]:
words = list(np.asarray(Tfidf_vectorizer.get_feature_names()))

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [58]:
topic_model_c = ct.Corex(n_hidden=4, words=words, seed=1)
topic_model_c.fit(doc_word2, words=words, docs=df1['Review2'])

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [59]:
# Print all topics from the CorEx topic model
topics = topic_model_c.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: resort,beach,cana,punta,punta cana,food,vacat,pool,dominican,entertain
1: credit card,credit,card,charg,transact,cancel,account,statement,visa,debit
2: about,pride,problem breakfast,progress,promptli,proper,propos,prove,provid excel,provid nice
3: abil,provid servic,pull sofa,push,qualiti servic,queri,quick room,quiet area,quiet relax,quieter


/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 4- Latent Semantic Analysis (LSA) 

In [40]:
# model performing with 5 topic

lsa1_tf3 = TruncatedSVD(5)
doc_topic = lsa1_tf3.fit_transform(doc_word2)
display_topics(lsa1_tf3, Tfidf_vectorizer.get_feature_names(), 15)

/Users/alanoudabdulaziz/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic  0
hotel, room, great, good, staff, locat, nice, servic, beach, clean, breakfast, restaur, place, resort, food

Topic  1
resort, beach, pool, food, vacat, drink, water, cana, punta, punta cana, beauti, buffet, like, entertain, kid

Topic  2
great, locat, great locat, friendli, love, great hotel, hotel great, excel, staff, resort, recommend, beach, help, wonder, fantast

Topic  3
great, good, nice, great locat, locat, room, valu, small, clean, street, park, floor, free, bathroom, price

Topic  4
good, station, excel, hotel, valu, euro, metro, breakfast, barcelona, good valu, minut, resort, train, hotel good, english
